In [ ]:
import pandas as pd
import numpy as np
import os

from matplotlib import rcParams, pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import re
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Conv1D, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.snowball import SnowballStemmer
import warnings 

from matplotlib import pyplot as plt
from matplotlib import rcParams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import seaborn as sns
warnings.filterwarnings(action='ignore')

rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/데이콘 - 소셜 작가 분류/open/train.csv')
test = pd.read_csv('/content/drive/MyDrive/데이콘 - 소셜 작가 분류/open/test_x.csv')

In [ ]:
# 작가 분포 확인
train['author'].value_counts()

3    15063
0    13235
2    11554
4     7805
1     7222
Name: author, dtype: int64

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
else:
    print('No GPU detected')

No GPU detected


In [ ]:
data_dir = Path('/content/drive/MyDrive/데이콘 - 소셜 작가 분류/open')
feature_dir = Path('/content/drive/MyDrive/데이콘 - 소셜 작가 분류/feature')
val_dir = Path('/content/drive/MyDrive/데이콘 - 소셜 작가 분류/val')
tst_dir = Path('/content/drive/MyDrive/데이콘 - 소셜 작가 분류/tst')
sub_dir = Path('/content/drive/MyDrive/데이콘 - 소셜 작가 분류/sub')
dirs = [feature_dir, val_dir, tst_dir, sub_dir]
for d in dirs:
    os.makedirs(d, exist_ok=True)

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

In [ ]:
# 불용어 제거
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)


def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)


stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [ ]:
train['text'] = train['text'].str.lower().apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].str.lower().apply(alpha_num).apply(remove_stopwords)

In [ ]:
X_train = train['text'].values
X_test = test['text'].values
y = train['author'].values
print(X_train.shape, X_test.shape, y.shape)

(54879,) (19617,) (54879,)


In [ ]:
# 자연어를 모델이 이해하기 쉽도록 만드는 것이 vectorize 방식인데
# 벡터화한 고유의 토큰들이 모인 집합을 vocabulary라고 함
# 이때 vocab_size가 크면 클수록 학습하는데 오래걸리게 됨
vocab_size = 20000
embedding_dim = 64
max_length = 500
padding_type='post'

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [ ]:
# 텍스트 데이터를 연속된 벡터값으로 변환 시켜주는 것
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
trn = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)
tst = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)
print(trn.shape, tst.shape)


(54879, 500) (19617, 500)


In [ ]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
# DNN모델
def get_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Dropout(.5),
        Conv1D(128, 7, padding="valid", activation="relu", strides=3),
        Conv1D(128, 7, padding="valid", activation="relu", strides=3),    
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(.5),
        Dense(n_class, activation='softmax')
    ])
    
    # compile model
    # model.compile() 안에서는 optimizer, loss, metric를 
    # 모델을 작동시킬 때 설정한 metric으로 로그를 찍어줄 수 있음
    # loss는 자동저장
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=.005))
    return model

In [ ]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)

    clf = get_model()    
    clf.fit(trn[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(trn[i_val], to_categorical(y[i_val])),
            epochs=10,
            batch_size=512,
            callbacks=[es])
    p_val[i_val, :] = clf.predict(trn[i_val])
    p_tst += clf.predict(tst) / n_fold

training model for CV #1
Epoch 1/10
86/86 [==============================] - 131s 2s/step - loss: 1.3846 - val_loss: 1.1078
Epoch 2/10
86/86 [==============================] - 131s 2s/step - loss: 0.9856 - val_loss: 0.8763
Epoch 3/10
86/86 [==============================] - 132s 2s/step - loss: 0.7197 - val_loss: 0.8047
Epoch 4/10
86/86 [==============================] - 131s 2s/step - loss: 0.5723 - val_loss: 0.8015
Epoch 5/10
86/86 [==============================] - 132s 2s/step - loss: 0.4806 - val_loss: 0.8346
Epoch 6/10
86/86 [==============================] - 131s 2s/step - loss: 0.4175 - val_loss: 0.8690
Epoch 7/10
86/86 [==============================] - 131s 2s/step - loss: 0.3745 - val_loss: 0.9246
Epoch 00007: early stopping
training model for CV #2
Epoch 1/10
86/86 [==============================] - 130s 2s/step - loss: 1.3105 - val_loss: 1.0188
Epoch 2/10
86/86 [==============================] - 132s 2s/step - loss: 0.8959 - val_loss: 0.8593
Epoch 3/10
86/86 [=============

In [ ]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p_val, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p_val):8.4f}')

Accuracy (CV):  71.2039%
Log Loss (CV):   0.7894


In [ ]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p_val, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p_val):8.4f}')

Accuracy (CV):  71.2039%
Log Loss (CV):   0.7894


In [ ]:
np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

NameError: ignored

In [ ]:
sub = pd.read_csv('/content/drive/MyDrive/데이콘 - 소셜 작가 분류/open/sample_submission.csv', index_col=0)
print(sub.shape)
sub.head()

(19617, 5)


,0,1,2,3,4
index,,,,,
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [ ]:
sub[sub.columns] = p_tst
sub.head()

,0,1,2,3,4
index,,,,,
0,0.1506,0.2496,0.3269,0.2207,0.0523
1,0.1614,0.6608,0.0660,0.0221,0.0897
2,0.9732,0.0112,0.0016,0.0001,0.0139
3,0.0284,0.0069,0.7091,0.0042,0.2515
4,0.5993,0.0683,0.1405,0.0408,0.1510


In [ ]:
sub.to_csv('/content/drive/MyDrive/데이콘 - 소셜 작가 분류/open/submission_ver1.csv')